In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os 

In [2]:
import torch

if torch.cuda.is_available():
    print("GPU .")
else:
    print("CPU.")

GPU .


In [3]:
# Définir les transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Redimensionner les images à 128x128
    transforms.ToTensor()           # Convertir les images en tenseurs PyTorch
])

In [4]:
test_data = datasets.ImageFolder('dataset/test',transform=transform)

In [5]:
test_loader = DataLoader(test_data, batch_size=32, shuffle=False,pin_memory=True,num_workers=os.cpu_count())

In [6]:
len(test_data.classes)

525

In [7]:
class ImageCNN(nn.Module):
    def __init__(self,num_classes):
        super(ImageCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)  
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)   
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)  
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)    
        self.bn4 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.bn4 = nn.BatchNorm2d(256)
        self.dropout = nn.Dropout(p=0.5)

        
        self.fc1 = nn.Linear(256 * 8 * 8, 1024)  # Updated for more neurons due to more convolution layers
        self.fc2 = nn.Linear(1024, num_classes)
        

    def forward(self, x):    #32,3,128,128 
        x = self.dropout(self.bn1(F.relu(self.pool(self.conv1(x))))) #32,32,64,64
        x = self.dropout(self.bn2(F.relu(self.pool(self.conv2(x)))) )#32,64,32,32
        x = self.dropout(self.bn3(F.relu(self.pool(self.conv3(x))))) #32,128,16,16
        x = self.dropout(self.bn4(F.relu(self.pool(self.conv4(x))))) #32,256,8,8
        x = x.view((len(x),-1))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

In [8]:
# Déplacer le modèle sur le GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [9]:

# Charger le modèle
model =ImageCNN(525)
model.load_state_dict(torch.load('model.pth'))
model.to(device)
model.eval()  # Mettre le modèle en mode évaluation


ImageCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=16384, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=525, bias=True)
)

In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
# Calculer la perte et la précision sur le jeu de validation
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total
print(f" Perte de test: {test_loss:.4f}, Précision de test: {test_accuracy:.2f}%")




 Perte de test: 1.6264, Précision de test: 64.23%
